In [1]:
!pip install tensorflow

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [8]:
# =======================
# ANN MODEL (CLASSIFICATION)
# =======================

!pip install tensorflow

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 1️⃣ Load Dataset
df = pd.read_excel("Dataset_(Task-1).xlsx")

# Clean numeric columns
for col in df.columns:
    if col != "Bed Form":
        df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna()

# 2️⃣ Split Features & Target
X = df.drop("Bed Form", axis=1)
y = df["Bed Form"]

# Encode target
encoder = LabelEncoder()
y = encoder.fit_transform(y)

# 3️⃣ Train Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 4️⃣ Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 5️⃣ Build ANN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

# 6️⃣ Compile
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 7️⃣ Train
model.fit(X_train, y_train, epochs=30, batch_size=32)

# 8️⃣ Evaluate
loss, acc = model.evaluate(X_test, y_test)
print("ANN Accuracy:", acc)

Epoch 1/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3506 - loss: 1.3580
Epoch 2/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9602 - loss: 0.6398
Epoch 3/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9591 - loss: 0.2789
Epoch 4/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9580 - loss: 0.2212
Epoch 5/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9646 - loss: 0.1788
Epoch 6/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9509 - loss: 0.1847
Epoch 7/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9679 - loss: 0.1191
Epoch 8/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9616 - loss: 0.1408
Epoch 9/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9655 - loss: 0.1075
Epoch 10/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9675 - loss: 0.1069
Epoch 11/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9673 - loss: 0.1045
Epoch 12/30
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9644 - lo

In [9]:
# =======================
# PINN MODEL
# =======================

import tensorflow as tf
import numpy as np

class PINN(tf.keras.Model):
    def __init__(self, output_dim):
        super(PINN, self).__init__()
        self.d1 = tf.keras.layers.Dense(32, activation='relu')
        self.d2 = tf.keras.layers.Dense(16, activation='relu')
        self.out = tf.keras.layers.Dense(output_dim, activation='softmax')

    def call(self, x):
        x = self.d1(x)
        x = self.d2(x)
        return self.out(x)

# Create model
pinn = PINN(output_dim=len(np.unique(y)))
optimizer = tf.keras.optimizers.Adam()

# Convert to tensor
X_train_tf = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train_tf = tf.convert_to_tensor(y_train, dtype=tf.int32)

# Custom loss
def pinn_loss(model, X, y_true):

    y_pred = model(X)

    # Data loss
    data_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    # Physics constraint (example)
    flow_depth = X[:, 0]
    slope = X[:, 1]
    charge = X[:, 2]

    physics_eq = slope * flow_depth
    physics_loss = tf.reduce_mean(tf.square(charge - physics_eq))

    return tf.reduce_mean(data_loss) + 0.1 * physics_loss

# Training loop
epochs = 30

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = pinn_loss(pinn, X_train_tf, y_train_tf)

    grads = tape.gradient(loss, pinn.trainable_variables)
    optimizer.apply_gradients(zip(grads, pinn.trainable_variables))

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, PINN Loss: {loss.numpy()}")

Epoch 0, PINN Loss: 1.5049995183944702
Epoch 5, PINN Loss: 1.4348840713500977
Epoch 10, PINN Loss: 1.367566466331482
Epoch 15, PINN Loss: 1.301991581916809
Epoch 20, PINN Loss: 1.2373385429382324
Epoch 25, PINN Loss: 1.1729545593261719


In [12]:
# Get predictions
y_pred = pinn(X_test_tf)

# Convert probabilities to predicted class labels
y_pred_classes = tf.argmax(y_pred, axis=1)

# Calculate accuracy
accuracy = tf.reduce_mean(
    tf.cast(y_pred_classes == y_test_tf, tf.float32)
)

print("PINN Accuracy:", accuracy.numpy())

PINN Accuracy: 0.9647059
